# [TGS Salt Identification Challenge](https://www.kaggle.com/c/tgs-salt-identification-challenge)
by: frank@aixpact.com

## Changelog

- experimenting with cumsum and depth
  - did not improve the scores
  - will explore data augmenting and EDA in seperate notebook


## Credits to

I based this notebook and insights on:

[Alexander Liao](https://www.kaggle.com/alexanderliao/u-net-bn-aug-strat-dice/notebook)  -- Dice and augmentation  
[Jesper Dramsch](https://www.kaggle.com/jesperdramsch/intro-to-seismic-salt-and-how-to-geophysics) -- About session. 

## About

Seismic data is a neat thing. You can imagine it like an ultra-sound of the subsurface. However, in an ultra-sound, we use much smaller wavelengths to image our body. Seismic data usually has wavelengths around 1m to 100m. That has some physical implications, but for now, we don't have to deal with that. It's just something to keep in mind while thinking about resolution. 

Imaging salt has been a huge topic in the seismic industry, basically since they imaged salt the first time. The Society of Exploration geophysicist alone has over 10,000 publications with the [keyword salt](https://library.seg.org/action/doSearch?AllField=salt). Salt bodies are important for the hydrocarbon industry, as they usually form nice oil traps. So there's a clear motivation to delineate salt bodies in the subsurface. If you would like to do a deep dive, you can see [this publication](https://www.iongeo.com/content/documents/Resource%20Center/Articles/INT_Imaging_Salt_tutorial_141101.pdf)

Seismic data interpreters are used to interpreting on 2D or 3D images that have been heavily processed. The standard work of [seismic data analysis](https://wiki.seg.org/wiki/Seismic_Data_Analysis) is open access.
You'll find sections on Salt in there as well (https://wiki.seg.org/wiki/Salt-flank_reflections and https://wiki.seg.org/wiki/Salt_flanks). The seismic itself is pretty "old" in the publication, and you're dealing with data that is less noisy here, which is nice.

[![Seismic Data with salt CC-BY-SA Yilmaz](https://wiki.seg.org/images/1/14/Ch05_fig0-1.png)](https://wiki.seg.org/wiki/Salt-flank_reflections#/media/File:Ch05_fig0-1.png)
Caption: Figure 5.0-1  Conflicting dips associated with salt flanks: (a) CMP stack without dip-moveout correction; (b) time migration of the stack in (a); (c) the stack with dip-moveout correction; (d) time migration of the stack in (c). CC-BY-SA Yilmaz.

Interpretation on seismic images has long used texture attributes, to identify better and highlight areas of interest. These can be seen like feature maps on the texture of the seismic. For salt, you will notice that the texture in the salt masks is rather chaotic, where the surrounding seismic is more "striped". You can think of Earth as layered. Sand gets deposited on top of existing sand. In comes salt, which is behaving very much, unlike other rocks. There is an entire research branch dedicated to salt tectonics, that is the movement of salt in the subsurface. To give you the gist, these salt diapirs form from salt layers somewhere else that were under much pressure. These started to flow (behave ductile) and find a way into other layers above. I have written a bit about salt on [my blog](http://the-geophysicist.com/the-showroom-data-for-my-thesis).

One common seismic attribute is called "chaos" or "seismic disorder". So if you talk to cynic geophysicists, you'll hear "that deep learning better outperform the Chaos attribute". A good starting point is [this publication](http://www.chopraseismic.com/wp-content/uploads/2016/08/Chopra_Marfurt_TLE_Aug2016-LowRes.pdf).

Recently, geoscience has started to adopt deep learning, and it has seen a clear boom, particularly in imaging salt. Code for automatic seismic interpretation can be found here: 

+ https://github.com/waldeland/CNN-for-ASI
+ https://github.com/bolgebrygg/MalenoV
+ https://github.com/crild/facies_net

You will notice that these solutions load a specific SEG-Y file, which luckily we don't have to bother with. TGS provided some nice PNG files instead. However, you can glean some information from them how to approach seismic data. If you find you need some geophysical helpers, you can [import Bruges](https://github.com/agile-geoscience/bruges)


## Setup notebook

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import seaborn as sns
sns.set_style("white")
%matplotlib inline

# import cv2

from tqdm import tqdm_notebook, tnrange

from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize

from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Dropout, BatchNormalization, UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose

from keras.optimizers import Adam, SGD
from keras.losses import binary_crossentropy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.vis_utils import plot_model
from keras import backend as K

### Set file locations

In [ ]:
path = './input'
path_train = f'{path}/train'
path_test = f'{path}/test'
imgs_train = f'{path}/train/images'
masks_train = f'{path}/train/masks'
imgs_test = f'{path}/test/images'

### Set/initiate constants

In [ ]:
IMG_SIZE = 101
TGT_SIZE = 128
MAX_DEPTH = None # set after loading data

In [ ]:
def upsample(img, img_size_target=TGT_SIZE):
    """Resize image to target shape"""
    img_size = img.shape[0]
    if img_size == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True)


def downsample(img, img_size_orig=IMG_SIZE):
    """Resize image to original shape"""
    img_size = img.shape[0]
    if img_size == img_size_orig:
        return img
    return resize(img, (img_size_orig, img_size_orig), mode='constant', preserve_range=True)


def imgs_2_array(path, img_names, ftype='.png', size=TGT_SIZE):
    """Load images as arrays in target shape: (-1, target, target, 1)"""
    imgs = [upsample(np.array(load_img(f'{path}/{name}{ftype}', grayscale=True))) / 255
                      for name in tqdm_notebook(img_names)]
    imgs.extend([np.fliplr(img) for img in imgs]) # extend set with flips
    imgs = np.array(imgs).reshape(-1, size, size, 1)
    return imgs
        
    
def csum(img, border=2):
    """Create image cumsum from image
    Sort of image bleeding downwards"""
    center_mean = img[border:-border, border:-border].mean()
    csum = (np.float32(img)-center_mean).cumsum(axis=0)         
    csum -= csum[border:-border, border:-border].mean()
    csum /= max(1e-3, csum[border:-border, border:-border].std())
    return csum


def imgs_2_csum(path, img_names, ftype='.png', size=TGT_SIZE):
    """Load images and transform to array with image and cumsum layer"""
    imgs = imgs_2_array(path, img_names, ftype, size)
    img_csums = [csum(img) for img in tqdm_notebook(imgs)]
    return np.concatenate((np.array(imgs), np.array(img_csums)) , axis=3)

## Data Exploration

Load and build train and test set.

 - contemplating to use depth as a layer, this explains the stacking in `images_d`.
 - checking depth
 - checking coverage
 - checking depth - coverage relationship
 - visualize seismic images with masks

In [ ]:
train_df_ = pd.read_csv(f'{path}/train.csv', index_col="id", usecols=[0])
depths_df_ = pd.read_csv(f'{path}/depths.csv', index_col="id")
train_df_ = train_df_.join(depths_df_)
train_indices = train_df_.index

# Flip(augment) train images -> duplicate df: images and depth
train_df = pd.concat([train_df_, train_df_])

test_df = depths_df_[~depths_df_.index.isin(train_indices)]

In [ ]:
train_df.info()

In [ ]:
imgs = imgs_2_array(imgs_train, train_indices, '.png', TGT_SIZE)
train_df["images"] = [img for img in imgs]
train_df["images"][0].shape

In [ ]:
imgs = imgs_2_csum(imgs_train, train_indices, '.png', TGT_SIZE)
train_df["images_d"] = [img for img in imgs]
train_df["images_d"][0].shape

In [ ]:
masks = imgs_2_array(masks_train, train_indices, '.png', TGT_SIZE)
train_df["masks"] = [mask for mask in masks]
train_df["masks"][0].shape

In [ ]:
depth = train_df["z"]
mean_depth, std_depth, max_depth = depth.mean(), depth.std(), depth.max()
norm_depth = (depth - mean_depth) / std_depth
train_df["depth"] = norm_depth
train_df["depth"].head()

In [ ]:
# Sanity check
train_df[:5]
train_df[4000:4005]

### Depth

 - checking the distribution
 
*As per below: depth is 'normal' distributed and train and test set have same distribution.*

In [ ]:
_ = sns.distplot(train_df.z, label="Train")
_ = sns.distplot(test_df.z, label="Test")
_ = plt.legend()
_ = plt.title("Depth distribution")

### Mask coverage

*As per below: there are 8-10 times more seismic images with 0-10% salt areas. Stratification in train-validation split must prevent overfitting.*

In [ ]:
def coverage(mask):
    """Compute salt mask coverage"""
    return np.sum(mask) / (mask.shape[0]*mask.shape[1])


def norm_coverage(masks):
    """Compute salt mask coverage"""
    coverages = masks.map(coverage)
    mean_cov, std_cov, max_cov = coverages.mean(), coverages.std(), coverages.max()
    return(coverages - mean_cov) / std_cov


def coverage_class(mask):
    """Compute salt mask coverage class"""
    if coverage(mask) == 0:
            return 0
    return (coverage(mask) * 100 //10).astype(np.int8) +1

In [ ]:
_ = sns.distplot(train_df.masks.map(coverage_class), label="Train", kde=False)
_ = plt.legend()
_ = plt.title("Coverage distribution")

## Depth vs. coverage

 - checking the correlation between depth and coverage
 
*As per below: no pattern or correlation visible, depth and coverage are unrelated. Depth might still be a factor in the prediction, e.g. the structure/grain might relate to depth.*

In [ ]:
# salt_cover = train_df.masks.map(coverage)
salt_cover = norm_coverage(train_df.masks)
# depth = train_df.z / MAX_DEPTH

In [ ]:
salt_cover = norm_coverage(train_df.masks)

plt.figure(figsize=(20,10))
plt.scatter(range(len(norm_depth)), norm_depth, alpha=.5)
plt.scatter(range(len(salt_cover)), salt_cover, color='r', alpha=.5)
plt.title('Normalized:\nDepth as % of maximum depth(blue) vs. Salt coverage as % of image size(red)', fontsize=20)
plt.show();

## Visualize images and masks (overlayed)

In [ ]:
def plot_imgs_masks(imgs, masks, preds_valid=None, thres=None, grid_width=10, zoom=1.5):
    """Visualize seismic images with their salt area mask(green) and optionally salt area prediction(orange). 
    The prediction mask can be either in float-mask or binary-mask form(based on threshold)
    """
    grid_height = 1 + (len(imgs)-1) // grid_width
    fig, axs = plt.subplots(grid_height, grid_width, figsize=(grid_width*zoom, grid_height*zoom))
    axes = axs.ravel()
    
    for i, img in enumerate(imgs):
        mask = masks[i]
        depth, MAX_DEPTH = 1, 1 # img[0, 0, 1] # TODO
        
        
        ax = axes[i] #//grid_width, i%grid_width]
        _ = ax.imshow(img[..., 0], cmap="Greys")
        _ = ax.imshow(img[..., 1], alpha=0.15, cmap="seismic") # TODO
        _ = ax.imshow(mask[..., 0], alpha=0.3, cmap="Greens")
        
        if preds_valid is not None:
            if thres is not None:
                pred = np.array(np.round(preds_valid[i] > thres), dtype=np.float32)
            else:
                pred = preds_valid[i]
            _ = ax.imshow(pred, alpha=0.3, cmap="OrRd")
        
        _ = ax.text(2, img.shape[0]-2, depth * MAX_DEPTH//1, color="k")
        _ = ax.text(img.shape[0]-2, 2, round(coverage(mask), 2), color="k", ha="right", va="top")
        _ = ax.text(2, 2, coverage_class(mask), color="k", ha="left", va="top")
        
        _ = ax.set_yticklabels([])
        _ = ax.set_xticklabels([])
        _ = plt.axis('off')
    plt.suptitle("Green: Salt area mask \nTop-left: coverage class, top-right: salt coverage, bottom-left: depth", y=1+.5/grid_height, fontsize=20)
    plt.tight_layout();

In [ ]:
N = 40
plot_imgs_masks(train_df.iloc[:N].images_d, train_df.iloc[:N].masks)

## Train test split

In [ ]:
ids_train, ids_valid, x_train, x_valid, y_train, y_valid, depth_train, depth_valid = train_test_split(
    train_df.index.values,
    np.array(train_df.images_d.map(upsample).tolist()).reshape(-1, TGT_SIZE, TGT_SIZE, 2), # FJE images_d
    np.array(train_df.masks.map(upsample).tolist()).reshape(-1, TGT_SIZE, TGT_SIZE, 1), 
    train_df.depth.values,
    test_size=0.15, 
    stratify=train_df.masks.map(coverage_class), 
    random_state=1)

#### Sanity check 

In [ ]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

We have many examples without salt, as you can see by the masks that are entirely dark. That's great, an algorithm we build will then know that patches exist entirely without salt. Talk about biasing your data.

We can draw heavily on other work, instead of regurgitating the geophysics work that has been done before. I mentioned that seismic is kind of like ultrasound. So I had a look at https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

Let's throw a Unet at our data. I am blatanly stealing from Ketil at this point. All credit goes to him and his nice code.
First we'll need to get our data into a shape that works for U-Nets. That means, it should be a power of 2. Let's do it quick and dirty for now, but eventually, consider aliasing and all that fun.

## Data augmentation and preprocessing

To address overfitting, to increase the training set and as result to boost performance.

### Model constants

In [ ]:
BATCH_SIZE = 16
EPOCHS = 5
# LAST_BATCH = len(x_train)%BATCH_SIZE
# N_BATCHES = int(LAST_BATCH > 0) + len(x_train)//BATCH_SIZE
# print(LAST_BATCH, N_BATCHES)

### Data generator

As the mask is related to the image, both the mask and the image must be augmented identically, by creating identical generators and using the same seed.

**Note:**  
 - Shifting alters the edges and does more harm than good.  
 - Zoom-out alters the edges and does more harm than good. 
 - Vertical flip seems to halt the loss/preformance from start.
 - Standardization changes mask boolean values to negative values, this will raise error at training.

In [ ]:
# Create two instances with the same arguments
data_gen_args = dict(
    rotation_range=2.,
    zoom_range=[.8, 1],
    horizontal_flip=True,
    )

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Provide the same seed and keyword arguments to the fit and flow methods
image_datagen.fit(x_train, seed=1)
mask_datagen.fit(y_train, seed=1)

image_generator = image_datagen.flow(
    x_train,
    batch_size=BATCH_SIZE,
    seed=1)

mask_generator = mask_datagen.flow(
    y_train,
    batch_size=BATCH_SIZE,
    seed=1)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

### Generate augmented dataset

Set the size of the training set as you like, generating(bootstrapping) more augmented samples

In [ ]:
X_train_l = []
Y_train_l = []
    
# add examples to list by batch
for batch_id, (x_batch, y_batch) in tqdm_notebook(enumerate(train_generator)):
    # Add full batches only - prevent odd array shapes
    if x_batch.shape[0] == BATCH_SIZE:
        X_train_l.append(x_batch)
        Y_train_l.append(y_batch)
    # Break infinite loop manually when required number of batches is reached
    if len(X_train_l) == N_BATCHES: break

# Sanity check all arrays are same shape
assert len(set(arr.shape for arr in X_train_l)) == 1
assert len(set(arr.shape for arr in Y_train_l)) == 1

# Stack list of arrays
X_train_augm = np.vstack(X_train_l)
Y_train_augm = np.vstack(Y_train_l)

# Sanity check stacking over first dimension
assert X_train_augm.shape[0] == BATCH_SIZE * N_BATCHES
assert Y_train_augm.shape[0] == BATCH_SIZE * N_BATCHES

print('Done!')

In [ ]:
X_train_augm[:60].shape
Y_train_augm[:60].shape

## Visualize augmented data

In [ ]:
N = 60
plot_imgs_masks(X_train_augm[:N], Y_train_augm[:N])

## Build model



In [ ]:
def mean_iou(y_true, y_pred, score_thres=0.5):
    """Compute mean-IoU metric(mean-intersect-over-union-score).
    
    For each (mask)threshold in provided range:
     - create boolean mask (from float mask) based on threshold
     - score the mask 1 if IoU > score_threshold(0.5)
    Take the mean of the scores

    https://www.tensorflow.org/api_docs/python/tf/metrics/mean_iou
    """
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_bool = tf.to_int32(y_pred > t) # boolean mask by threshold
        score, update_op = tf.metrics.mean_iou(y_true, y_pred_bool, 2) # mean score over batch(=1)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
#             tf.cast(score > score_thres, tf.int32) # provided threshold for iou-score
            score = tf.identity(score)
        prec.append(score)
        
    return K.mean(K.stack(prec), axis=0)

### Model architecture

In [ ]:
def conv_block(m, ch_dim, acti, bn, res, do=0):
    """CNN block"""
    n = Conv2D(ch_dim, 3, activation=acti, padding='same')(m)
    n = BatchNormalization()(n) if bn else n
    n = Dropout(do)(n) if do else n
    n = Conv2D(ch_dim, 3, activation=acti, padding='same')(n)
    n = BatchNormalization()(n) if bn else n
    return Concatenate()([m, n]) if res else n

In [ ]:
def input_feature(f, n):
    """"""
    # We put the depth somewhere in the middle
    nn, features = 1, 1
    xx       = K.int_shape(n)[1]
    f_repeat = RepeatVector(xx*xx*nn)(f)
    f_conv   = Reshape((xx,xx,nn*features))(f_repeat)
    n        = Concatenate(axis=-1, name=f'feat_{2}')([n, f_conv])
    n        = BatchNormalization()(n)            
#     n        = Activation('relu')(n) ## <-------------- May be
    return n

In [ ]:
def level_block(m, input_depth, inp_feat, ch_dim, depth, inc_rate, acti, do, bn, mp, up, res):
    """Recursive CNN builder"""
    featureOK = True
    if depth > 0:
        n = conv_block(m, ch_dim, acti, bn, res)
        m = MaxPooling2D()(n) if mp else Conv2D(ch_dim, 3, strides=2, padding='same')(n)
        if featureOK and (depth==DEPTH-1):
            m = Concatenate()([m, input_feature(inp_feat, m)])
        m = level_block(m, input_depth, inp_feat, int(inc_rate*ch_dim), depth-1, inc_rate, acti, do, bn, mp, up, res)
        if up:
            m = UpSampling2D()(m)
            m = Conv2D(ch_dim, 2, activation=acti, padding='same')(m)
        else:
            m = Conv2DTranspose(ch_dim, 3, strides=2, activation=acti, padding='same')(m)
        # 
        n = Concatenate()([n, m])
        m = conv_block(n, ch_dim, acti, bn, res)
    else:
        # lowest/middle block
        m = conv_block(m, ch_dim, acti, bn, res, do)
        m = Concatenate()([m, input_depth])
    return m

In [ ]:
DPT_SIZE = 4

def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
        dropout=0.5, batchnorm=False, maxpool=True, upconv=False, residual=False):
    """Returns model"""
    inputs = Input(shape=img_shape, name='img')
    input_depth = Input(shape=(DPT_SIZE, DPT_SIZE, 1), name='depth')
    inp_feat = Input(shape=(1,), name='feat')
    outputs = level_block(inputs, input_depth, inp_feat, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    outputs = Conv2D(out_ch, 1, activation='sigmoid')(outputs)
    return Model(inputs=[inputs, input_depth, inp_feat], outputs=outputs)

In [ ]:
# def UNet(img_shape, out_ch=1, start_ch=64, depth=4, inc_rate=2., activation='relu', 
#         dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
#     """Returns model"""
#     inputs = Input(shape=img_shape)
#     outputs = level_block(inputs, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
#     outputs = Conv2D(out_ch, 1, activation='sigmoid')(outputs)
#     return Model(inputs=inputs, outputs=outputs)

In [ ]:
DEPTH = 5
model = UNet((TGT_SIZE, TGT_SIZE, 2), 
             start_ch=16, 
             depth=DEPTH, 
             batchnorm=True)

In [ ]:
adam = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay=0.01)

model.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy", mean_iou])
# model.compile(loss=bce_dice_loss, optimizer="adam", metrics=["accuracy"])

In [ ]:
model.summary()

## Train model

In [ ]:
callbacks = [
    EarlyStopping(patience=7, verbose=1),
    ReduceLROnPlateau(patience=3, verbose=1),
    ModelCheckpoint('model-tgs-salt-1.h5', verbose=1, save_best_only=True)]
#                     save_weights_only=True)

history = model.fit({'img': x_train, 
                     'feat': depth_train, 
                     'depth': np.ones((x_train.shape[0], 4, 4, 1))}, # only use first layer
                    y_train, 
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    epochs=1, #EPOCHS, # only test 1st batch
                    callbacks=callbacks)


# Epoch 20/50
# 5440/5440 [==============================] - 32s 6ms/step 
# - loss: 0.1346 
# - mean_iou: 0.6909 
# - val_loss: 0.2147 
# - val_mean_iou: 0.6940

In [ ]:
history.history.keys()

In [ ]:
fig, (ax_loss, ax_acc, ax_iou) = plt.subplots(1, 3, figsize=(15,5))

_ = ax_loss.plot(history.epoch, history.history["loss"], label="Train loss")
_ = ax_loss.plot(history.epoch, history.history["val_loss"], label="Validation loss")

_ = ax_acc.plot(history.epoch, history.history["acc"], label="Train accuracy")
_ = ax_acc.plot(history.epoch, history.history["val_acc"], label="Validation accuracy")

_ = ax_iou.plot(history.epoch, history.history["mean_iou"], label="Train IoU")
_ = ax_iou.plot(history.epoch, history.history["val_mean_iou"], label="Validation IoU")

## Test Data

First we'll get the test data. This takes a while, it's 18000 samples.

In [ ]:
# Predict on train, val and test
model = load_model('model-tgs-salt-1.h5', custom_objects={'mean_iou': mean_iou})

In [ ]:
preds_valid = model.predict(x_valid[...,:1], verbose=1).reshape(-1, TGT_SIZE, TGT_SIZE)
preds_valid = np.array([downsample(x) for x in preds_valid])
y_valid_ori = np.array([train_df.loc[idx].masks for idx in ids_valid])

In [ ]:
N = 80
imgs = train_df.loc[ids_valid[:N]].images_d
masks = train_df.loc[ids_valid[:N]].masks
preds = preds_valid[:N]
plot_imgs_masks(imgs, masks, preds_valid)

## Scoring

Score the model and do a threshold optimization by the best IoU

In [ ]:
# src: https://www.kaggle.com/aglotero/another-iou-metric
def iou_metric(y_true_in, y_pred_in, print_table=False):
    labels = y_true_in
    y_pred = y_pred_in
    
    true_objects = 2
    pred_objects = 2

    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    print(f'area_true: {area_true}, area_pred: {area_pred}')
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    print(f'area_true: {area_true[0][0]}, area_pred: {area_pred[0][0]}, union: {union[0][0]}')
    
    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union
    print(f'iou: {iou}')
    print(f'iou: {iou[0][0]}, intersect: {intersection[0][0]}, union: {union[0][0]}')

    # Precision helper function
    def precision_at(threshold, iou):
        mask = iou > threshold
        tp = np.sum(mask, axis=1) == 1   # Correct objects
        fp = np.sum(mask, axis=0) == 0  # Missed objects
        fn = np.sum(mask, axis=1) == 0  # Extra objects
        tp, fp, fn = np.sum(tp), np.sum(fp), np.sum(fn)
        return tp, fp, fn

    # Loop over IoU thresholds
    prec = []
    if print_table:
        print("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        if (tp + fp + fn) > 0:
            p = tp / (tp + fp + fn)
        else:
            p = 0
        if print_table:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    
    if print_table:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    return np.mean(prec)

In [ ]:
def iou_metric_batch(y_true_in, y_pred_in):
    """Compute IoU batchwise"""
    batch_size = y_true_in.shape[0]
    return np.mean([iou_metric(y_true_in[b], y_pred_in[b]) for b in range(batch_size)])

In [ ]:
def _union(y_true, y_pred):
    """"""
    return sum((y_true.ravel() + y_pred.ravel()) > 0)
    
_union(y_valid_ori[0], np.int32(preds_valid[0] > .5))

In [ ]:
def _intersect(y_true, y_pred):
    """"""
    return sum(y_true.ravel() * y_pred.ravel())
    
_intersect(y_valid_ori[0], np.int32(preds_valid[0] > .5))

In [ ]:
def _iou(y_true, y_pred):
    """"""
    return _intersect(y_true, y_pred)/ max(1e-7, _union(y_true, y_pred))

In [ ]:
def _intersect(y_true, y_pred):
    """"""
    return sum(y_true.ravel() * y_pred.ravel())

def _union(y_true, y_pred):
    """"""
    return sum((y_true.ravel() + y_pred.ravel()) > 0)

def _iou(y_true, y_pred):
    """"""
    return _intersect(y_true, y_pred)/ max(1e-7, _union(y_true, y_pred))

def miou(y_true, y_pred, score_thres=.5):
    """"""
    score = []
    for mask_thres in np.arange(0.5, 1.0, 0.05):
        y_pred_bool = y_pred > mask_thres
        score.append(_iou(y_true, y_pred_bool)> score_thres)
    return np.mean(score)

In [ ]:
for i in range(5):
    y_true, y_pred = y_valid_ori[i], np.int32(preds_valid[i] > .5)
    f'area_true: {np.sum(y_true)}, area_pred: {np.sum(y_pred)}'
    f'miou: {miou(y_true, y_pred)}, intersect: {_intersect(y_true, y_pred)}, union: {_union(y_true, y_pred)}'
    iou_metric(y_true, y_pred);
    

In [ ]:
y_true, y_pred = y_valid_ori[i], np.int32(preds_valid[i] > .5)

def iou_pix(y_true, y_pred, thres_mask, thres_score):
    """IoU implemented as tp/(tp + fp + fn) at pixel level
    """
    y_true = y_true.ravel()
    y_pred = (y_pred > thres_mask).ravel()
    
    tp = sum(y_true * y_pred)
    tn = sum((y_true + y_pred) == 0)
    fp = sum(y_true < y_pred)
    fn = sum(y_true > y_pred)
    return int((tp / max(1e-6, (tp + fp + fn))) > thres_score)

iou_prec(y_true, y_pred, 0.5, 0.5)

In [ ]:
y_true, y_pred = y_valid_ori[i], np.int32(preds_valid[i] > .5)

def iou_msk(y_true, y_pred, thres_mask, thres_score):
    """IoU implemented as tp/(tp + fp + fn) at mask level
    """
    
    y_true = int(np.sum(y_true) > 0)
    y_pred = int(np.sum(y_pred > thres_mask) > 0)
    print(y_true, y_pred)
#     tp = sum(y_true * y_pred)
#     tn = sum((y_true + y_pred) == 0)
#     fp = sum(y_true < y_pred)
#     fn = sum(y_true > y_pred)
#     return int((tp / max(1e-6, (tp + fp + fn))) > thres_score)

iou_prec(y_true, y_pred, 0.5, 0.5)

In [ ]:
def mean_iou_pix(y_trues, y_preds, thres_score):
    """"""
    for i in tqdm_notebook(range(len(y_trues))):
        np.mean([iou_pix(y_trues[i], y_preds[i], thres_mask, thres_score) for thres_mask in np.linspace(0.1, 0.9, 40)])
# 
# mean_iou_pix(y_valid_ori, preds_valid, 0.5)

In [ ]:
def mean_iou_msk(y_trues, y_preds, thres_score):
    """"""
    for i in tqdm_notebook(range(len(y_trues))):
        print(y_trues[i], y_preds[i])
        np.mean([iou_msk(y_trues[i], y_preds[i], thres_mask, thres_score) for thres_mask in np.linspace(0.1, 0.9, 40)])
# 
# mean_iou_msk(y_valid_ori, preds_valid, 0.5)

In [ ]:
y_valid_ori[100].shape

In [ ]:
iou_metric_batch(y_valid_ori, np.int32(preds_valid > .5))

### Best threshold for best IoU score (submission)

In [ ]:
thresholds = np.linspace(0, 1, 50)
ious = np.array([iou_metric_batch(y_valid_ori, np.int32(preds_valid > threshold)) for threshold in tqdm_notebook(thresholds)])

In [ ]:
ious # TODO

In [ ]:
threshold_best_index = np.argmax(ious[9:-10]) + 9
iou_best = ious[threshold_best_index]
threshold_best = thresholds[threshold_best_index]

In [ ]:
_ = plt.plot(thresholds, ious)
_ = plt.plot(threshold_best, iou_best, "xr", label="Best threshold")
_ = plt.xlabel("Threshold")
_ = plt.ylabel("IoU")
_ = plt.title("Threshold vs IoU ({}, {})".format(threshold_best.round(), iou_best.round()))
_ = plt.legend()

In [ ]:
N = 80
imgs = train_df.loc[ids_valid[:N]].images_d
masks = train_df.loc[ids_valid[:N]].masks
preds = preds_valid[:N]
plot_imgs_masks(imgs, masks, preds_valid, threshold_best)

## Predict test set

### Sanity check if all indices and images match

In [ ]:
test_ids = next(os.walk(path_test+"/images"))[2]
test_ids[:3]

In [ ]:
set(test_ids) ^ set(test_df.index+'.png')

### Merge layers image and depth

In [ ]:
x_test = [upsample(np.array(load_img(f"{path_test}/images/{idx}.png", grayscale=True))) / 255 
                   for idx in tqdm_notebook(test_df.index)]

In [ ]:
x_test = np.array(x_test).reshape(-1, TGT_SIZE, TGT_SIZE)

In [ ]:
x_test_d = [np.ones_like(x_test[0]) * test_df.loc[i]["z"] / MAX_DEPTH
                     for i in tqdm_notebook(test_df.index)] 
x_test_d[0].shape

In [ ]:
x_test_m = np.dstack((np.array(x_test), np.array(x_test_d)))

In [ ]:
x_test_m = np.array([np.dstack((x_test[i], x_test_d[i])) for i, idx in tqdm_notebook(enumerate(test_df.index))])
x_test_m.shape

In [ ]:
preds_test = model.predict(x_test_m[..., :1]) # only use first layer

## Submission  

Submission is in csv form:
 - `id`: index (equals filename)
 - `rle_mask`: run-length format (down-then-right): `masked_pixel_start` `<space>` `length_of_masked_pixels` ...

In [ ]:
# Source https://www.kaggle.com/bguberfain/unet-with-depth
def RLenc(img, order='F', string=True):
    """Convert binary mask image to run-length array or string.
    
    pixel==1 locations are returned in format: <start length> ...
    
    Args:
    img: image in shape [n, m]
    order: is down-then-right, i.e. Fortran(F)
    string: return in string or array

    Return:
    run-length as an array or string
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if string:
        z = ''
        for rr in runs:
            z += f'{rr[0]} {rr[1]} '
        return z[:-1]
    else:
        return runs

In [ ]:
pred_dict = {idx: RLenc(np.round(downsample(preds_test[i]) > threshold_best)) 
             for i, idx in enumerate(tqdm_notebook(test_df.index.values))}

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict, orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')
sub.head()

### Nice job!

In [ ]:
5e-3

In [ ]:
def run_length_encode(text):
    '''
    Doctest:
        >>> encode('WWWWWWWWWWWWBWWWWWWWWWWWWBBBWWWWWWWWWWWWWWWWWWWWWWWWBWWWWWWWWWWWWWW')
        '12W1B12W3B24W1B14W'    
    '''
    from re import sub
    return sub(r'(.)\1*', lambda m: f'{text.index(m.group(0))} {len(m.group(0))} {"."} ', text)

In [ ]:
text = '0000111101011'
run_length_encode(text)

In [ ]:
def run_length_enc(text):
    """"""
    idx, rls = 0, ''
    text = text + '0'
    
    def RLE(text):
        """Run length encoding
        add '0' to start text to mark eof"""
        global idx, rls
        if idx==0: text += '0'
        start = text.index('1')
        idx += start
        end = text[start:].index('0')
        rls += f'{idx} {end} '
        idx += start
        print(rls)
        try:
#             print('*', text[start+end:])
            RLE(text[start+end:])
        except:
            print('done!')

    RLE(text)
    return rls

    

In [ ]:
def rle_encode(im, order='F'):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    Starts at position 1?
    '''
    pixels = img.reshape(img.shape[0] * img.shape[1], order=order)
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def rle_decode(rle_mask, shape=(101, 101), order='F'):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order=order)

In [ ]:
img = np.array([[0,1,1,1,0], [0,1,1,0,1], [0,1,1,1,0], [0,1,1,0,1], [0,1,1,0,1]])
img.flatten()

In [ ]:
pix = img.reshape(img.shape[0] * img.shape[1], order='F')
pix = np.concatenate([[0], pix, [0]])
pix
# runs = np.where(pix[1:] != pix[:-1])[0] + 1

In [ ]:
runs = np.where(pix[1:] != pix[:-1])[0] + 1 # index shifted by 1?
runs
runs[1::2] -= runs[::2]
runs
' '.join(str(x) for x in runs)

In [ ]:
pix[1:] * pix[:-1] != 0

In [ ]:
rle_encode(img)

In [ ]:
rle_decode(rle_encode(img), img.shape, order='F') == img

In [ ]:
rl

In [ ]:
def RLE(text, idx=0, rl=''):
    """Run length encoding
    add '0' to start text to mark eof"""
#         global idx, rl
    if idx==0: text += '0'
    start = text.index('1')
    idx += start
    end = text[start:].index('0')
    rl += f'{idx} {end} '
    idx += start
    print(rl)
    try:
#         print('*', text[start+end:])
        RLE(text[start+end:], idx, rl)
    except:
        print('done!')
RLE(text)